In [1]:
from balance import Sample
import pandas as pd
import numpy as np

INFO (2023-02-19 13:00:39,181) [__init__/<module> (line 52)]: Using balance version 0.4.0


In [2]:
df_train = pd.read_csv('../../data/processed/trainData.csv')
df_test = pd.read_csv('../../data/processed/testData.csv')

In [3]:
def preprocess_data(dataset):
    dataset = dataset.drop(['workclass','fnlwgt','education','marital-status','occupation','relationship','native-country'],axis=1)
    dataset['sex'] = np.where(dataset['sex'] == ' Male', 1, 0)
    dataset['race'] = np.where(dataset['race'] != ' White', 0, 1)
    dataset['Probability'] = np.where(dataset['Probability'] == ' <=50K', 0, 1)
    dataset['age'] = np.where(dataset['age'] >= 70, 70, dataset['age'])
    dataset['age'] = np.where((dataset['age'] >= 60 ) & (dataset['age'] < 70), 60, dataset['age'])
    dataset['age'] = np.where((dataset['age'] >= 50 ) & (dataset['age'] < 60), 50, dataset['age'])
    dataset['age'] = np.where((dataset['age'] >= 40 ) & (dataset['age'] < 50), 40, dataset['age'])
    dataset['age'] = np.where((dataset['age'] >= 30 ) & (dataset['age'] < 40), 30, dataset['age'])
    dataset['age'] = np.where((dataset['age'] >= 20 ) & (dataset['age'] < 30), 20, dataset['age'])
    dataset['age'] = np.where((dataset['age'] >= 10 ) & (dataset['age'] < 10), 10, dataset['age'])
    dataset['age'] = np.where(dataset['age'] < 10, 0, dataset['age'])

    return dataset

In [5]:
train_dataset = preprocess_data(df_train)
train_dataset['id'] = range(train_dataset.shape[0])
print(train_dataset)
test_dataset = preprocess_data(df_test)
test_dataset['id'] = range(train_dataset.shape[0], train_dataset.shape[0] + test_dataset.shape[0])
print(test_dataset)

       age  education-num  race  sex  capital-gain  capital-loss  \
0       50             10     1    0             0             0   
1       40             13     0    0             0             0   
2       40             13     0    0             0             0   
3       30              8     1    1             0             0   
4       30              8     1    0             0             0   
...    ...            ...   ...  ...           ...           ...   
36173   20              8     1    1             0             0   
36174   20             10     0    0             0             0   
36175   50             14     1    1             0             0   
36176   50              5     0    1             0             0   
36177   20              6     1    1             0             0   

       hours-per-week  Probability     id  
0                  49            0      0  
1                  35            1      1  
2                  60            0      2  
3      

In [7]:
sample = Sample.from_frame(train_dataset, outcome_columns=['Probability'], id_column='id')

target = Sample.from_frame(test_dataset)
sample_with_target = sample.set_target(target)

WARNING (2023-02-19 13:01:22,999) [sample_class/from_frame (line 187)]: Casting id column to string
/home/rohamkoohestani/anaconda3/lib/python3.9/site-packages/balance/util.py:1666: FutureWarning:

Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.

/home/rohamkoohestani/anaconda3/lib/python3.9/site-packages/balance/util.py:1666: FutureWarning:

Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.

WARNING (2023-02-19 13:01:23,137) [util/_warn_of_df_dtypes_change (line 1720)]: The dtypes of sample._df were changed from the original dtypes of the input df, here are the differences - 
WARNING (2023-02-19 13:01:23,138) [util/_warn_of_df_dtypes_change (line 1731)]: The (old) dtypes that changed for df (before the change):
WARNING (2023-02-19 13:01:23,141) [util/_warn_of_df_dtypes_change (line 1734)]: 
sex               int64
education-num     int64
race              int64
age               int64
hou

In [9]:
adjusted = sample_with_target.adjust(variables=['sex', 'race'])

INFO (2023-02-19 13:08:18,144) [ipw/ipw (line 428)]: Starting ipw function
WARNING (2023-02-19 13:08:18,145) [util/choose_variables (line 1154)]: Ignoring variables not present in all Samples: {'Probability'}
INFO (2023-02-19 13:08:18,152) [adjustment/apply_transformations (line 255)]: Adding the variables: []
INFO (2023-02-19 13:08:18,152) [adjustment/apply_transformations (line 256)]: Transforming the variables: ['sex', 'race']
INFO (2023-02-19 13:08:18,168) [adjustment/apply_transformations (line 293)]: Final variables in output: ['sex', 'race']
INFO (2023-02-19 13:08:18,203) [ipw/ipw (line 462)]: Building model matrix
INFO (2023-02-19 13:08:18,565) [ipw/ipw (line 486)]: The formula used to build the model matrix: ['sex + race']
INFO (2023-02-19 13:08:18,568) [ipw/ipw (line 489)]: The number of columns in the model matrix: 1
INFO (2023-02-19 13:08:18,569) [ipw/ipw (line 490)]: The number of rows in the model matrix: 45222
INFO (2023-02-19 13:08:19,092) [ipw/ipw (line 521)]: Fitting 

OSError: libgfortran.so.3: cannot open shared object file: No such file or directory